In [176]:
import pandas as  pd
import numpy as np
import csv
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

from nltk.stem import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


In [2]:
org = pd.read_csv(r"E:\Study\Sem 6\NLP\zomato.csv",dtype = str)
org.shape

(51717, 17)

In [3]:
org.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
org.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  object
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [5]:
df = org.drop(["url","dish_liked","phone"],axis = 1)

In [6]:
print("number of duplicated values:",df.duplicated().sum())
df.drop_duplicates(inplace=True)


number of duplicated values: 43


In [7]:
print("number of null values:\n",df.isnull().sum())
df.dropna(how='any',inplace=True)


number of null values:
 address                           0
name                              0
online_order                      0
book_table                        0
rate                           7767
votes                             0
location                         21
rest_type                       227
cuisines                         45
approx_cost(for two people)     345
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64


In [8]:
df = df.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})

In [9]:
df['cost'] = df['cost'].astype(str)
df['cost'] = df['cost'].apply(lambda x: x.replace(',',''))
df['cost'] = df['cost'].astype(float)


In [10]:
df.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [11]:
df.rate.unique()

array(['4.1/5', '3.8/5', '3.7/5', '3.6/5', '4.6/5', '4.0/5', '4.2/5',
       '3.9/5', '3.1/5', '3.0/5', '3.2/5', '3.3/5', '2.8/5', '4.4/5',
       '4.3/5', 'NEW', '2.9/5', '3.5/5', '2.6/5', '3.8 /5', '3.4/5',
       '4.5/5', '2.5/5', '2.7/5', '4.7/5', '2.4/5', '2.2/5', '2.3/5',
       '3.4 /5', '-', '3.6 /5', '4.8/5', '3.9 /5', '4.2 /5', '4.0 /5',
       '4.1 /5', '3.7 /5', '3.1 /5', '2.9 /5', '3.3 /5', '2.8 /5',
       '3.5 /5', '2.7 /5', '2.5 /5', '3.2 /5', '2.6 /5', '4.5 /5',
       '4.3 /5', '4.4 /5', '4.9/5', '2.1/5', '2.0/5', '1.8/5', '4.6 /5',
       '4.9 /5', '3.0 /5', '4.8 /5', '2.3 /5', '4.7 /5', '2.4 /5',
       '2.1 /5', '2.2 /5', '2.0 /5', '1.8 /5'], dtype=object)

In [12]:
df = df.loc[df.rate != 'NEW']
df = df.loc[df.rate != '-']

df.rate = df.rate.apply(lambda x: x.replace('/5','')).str.strip().astype(float)

df.rate.unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2, 2.3,
       4.8, 4.9, 2.1, 2. , 1.8])

In [137]:
zomato = df.copy()

In [138]:
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()

C:\Users\srich\AppData\Local\Temp/ipykernel_12012/368864937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()
C:\Users\srich\AppData\Local\Temp/ipykernel_12012/368864937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()


In [139]:
zomato.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
0,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,Banashankari,Casual Dining,"North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,4.118182
1,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,Banashankari,Casual Dining,"Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,4.100000
2,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,Banashankari,"Cafe, Casual Dining","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.800000
3,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,Banashankari,Quick Bites,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700000
4,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,Basavanagudi,Casual Dining,"North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.800000


In [140]:
zomato['Mean Rating'].max()

4.900000000000001

In [165]:
new_df = zomato.sample(10000)

In [166]:
zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
34493,"[('Rated 5.0', ""RATED\n Quite easy on the poc...","Biryani, North Indian"
10851,"[('Rated 5.0', 'RATED\n The parathas are the ...","South Indian, North Indian"
708,"[('Rated 3.0', ""RATED\n Couple of nights ago ...","Tibetan, Momos"
30547,"[('Rated 5.0', 'RATED\n Best place for Donne ...",Biryani
30106,"[('Rated 4.0', 'RATED\n Love the ambience and...","Cafe, North Indian, Chinese"


In [167]:
#remove remove sub ratings
new_df['reviews_list'] = new_df.reviews_list.str.replace(r"[0-9.]*[0-9]", " ")
#converting to lower case
new_df['reviews_list'] = new_df['reviews_list'].str.lower()
#removing punctuations
new_df['reviews_list'] = new_df.reviews_list.str.replace(r'[^\w\s]', ' ')

#removing urls
new_df['reviews_list'] = new_df.reviews_list.str.replace(r'https?://\S+|www\.\S+', ' ')

#removing stopwods and lemmatization
lem = WordNetLemmatizer()
#new_df['reviews_list'] = new_df['reviews_list'].apply(lambda x: " ".join([lem.lemmatize(word) for word in x.split() if word not in stop_words]))

C:\Users\srich\AppData\Local\Temp/ipykernel_12012/40896214.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['reviews_list'] = new_df.reviews_list.str.replace(r"[0-9.]*[0-9]", " ")
C:\Users\srich\AppData\Local\Temp/ipykernel_12012/40896214.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['reviews_list'] = new_df.reviews_list.str.replace(r'[^\w\s]', ' ')
C:\Users\srich\AppData\Local\Temp/ipykernel_12012/40896214.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df['reviews_list'] = new_df.reviews_list.str.replace(r'https?://\S+|www\.\S+', ' ')


In [168]:

# new_df["reviews_list"] = new_df["reviews_list"].replace("rated","",regex=False)


In [169]:
new_df["reviews_list"] = new_df["reviews_list"].str[23:]
new_df.head()

,address,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,menu_item,type,city,Mean Rating
42469,"Level 5, 1 MG Mall, Swami Vivekananda Road, Tr...",CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...,Yes,Yes,4.4,723,MG Road,"Casual Dining, Cafe","American, Cafe, Continental",1700.0,great food great taste superb presentation b...,"['Madras Chilli Butter Garlic Prawns', 'Non Ve...",Delivery,MG Road,4.400000
48275,"1, Cornwell Road, Next to St. Joseph's College...",Peace Restaurant,Yes,No,2.9,111,Shanti Nagar,Casual Dining,"Chinese, Tibetan, North Indian",500.0,we made a order from here delivery is quick ...,[],Dine-out,Residency Road,3.192308
20894,"38, Near ICICI Bank Signal, 4th F Cross, 8th M...",Durga Bikaner Sweets,No,No,3.7,15,Jayanagar,Quick Bites,"Street Food, Mithai",100.0,another famous spot in the jayanagar area whi...,[],Desserts,Jayanagar,3.700000
19818,"Tilak Nagar, Opposite Swagath Garuda Mall, Jay...",Fanoos Xprss,Yes,No,3.7,54,Jayanagar,Quick Bites,"Arabian, Rolls, North Indian, BBQ",400.0,sister to the brother on johnson market same ...,"['Chicken Minced Fanoos Roll', 'Mutton Seekh K...",Delivery,Jayanagar,3.700000
4723,"3/2, 1st Floor,Near Ramdev Medical, Bellandur ...",Ammi's Biryani,No,No,2.8,43,Bellandur,Quick Bites,"Biryani, North Indian",500.0,never again worest quality rated ra...,[],Dine-out,Bellandur,2.600000


In [170]:
restaurant_names = list(zomato['name'].unique())
restaurant_names[:5]

['Jalsa',
 'Spice Elephant',
 'San Churro Cafe',
 'Addhuri Udupi Bhojana',
 'Grand Village']

In [191]:
new_df = new_df.reset_index()

In [172]:
new_df.drop(['index','address','rest_type', 'type', 'menu_item', 'votes'],axis=1,inplace =True)


In [174]:
new_df.head()

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
0,CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...,Yes,Yes,4.4,MG Road,"American, Cafe, Continental",1700.0,great food great taste superb presentation b...,MG Road,4.400000
1,Peace Restaurant,Yes,No,2.9,Shanti Nagar,"Chinese, Tibetan, North Indian",500.0,we made a order from here delivery is quick ...,Residency Road,3.192308
2,Durga Bikaner Sweets,No,No,3.7,Jayanagar,"Street Food, Mithai",100.0,another famous spot in the jayanagar area whi...,Jayanagar,3.700000
3,Fanoos Xprss,Yes,No,3.7,Jayanagar,"Arabian, Rolls, North Indian, BBQ",400.0,sister to the brother on johnson market same ...,Jayanagar,3.700000
4,Ammi's Biryani,No,No,2.8,Bellandur,"Biryani, North Indian",500.0,never again worest quality rated ra...,Bellandur,2.600000


In [175]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0)
tfidf_matrix = tfidf.fit_transform(new_df['reviews_list'])

In [177]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [185]:
cosine_similarities

array([[1.        , 0.09290138, 0.15533194, ..., 0.23625388, 0.09527925,
        0.15149558],
       [0.09290138, 1.        , 0.05138709, ..., 0.09950672, 0.02384197,
        0.03792972],
       [0.15533194, 0.05138709, 1.        , ..., 0.1150553 , 0.04637211,
        0.08012055],
       ...,
       [0.23625388, 0.09950672, 0.1150553 , ..., 1.        , 0.06758067,
        0.12855276],
       [0.09527925, 0.02384197, 0.04637211, ..., 0.06758067, 1.        ,
        0.04806952],
       [0.15149558, 0.03792972, 0.08012055, ..., 0.12855276, 0.04806952,
        1.        ]])

In [194]:
temp = new_df                       #temp = df_percent
temp = temp.set_index("name")

In [197]:
indices = pd.Series(temp.index)

In [202]:
indices

0       CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ...
1                                        Peace Restaurant
2                                    Durga Bikaner Sweets
3                                            Fanoos Xprss
4                                          Ammi's Biryani
                              ...                        
9995                                       Smokie Brownie
9996                            Hyderabad Biryaani Palace
9997                                         The Wok Shop
9998                                      Sherlock's Cafe
9999                                      Cupcake Noggins
Name: name, Length: 10000, dtype: object

In [209]:
idx = indices[indices == "Ammi's Biryani"].index[0]
idx

4

In [247]:
def recommend_rest(indices, name, cos_sim):
    matches = []
    i = indices[indices == name].index[0]
    matches_indx = pd.Series(cos_sim[i]).sort_values(ascending = False)
    top_30 = list(matches_indx.iloc[:30].index)
    for i in top_30:
        matches.append(list(temp.index)[i])
    
    df_new = pd.DataFrame(columns=['cuisines', 'Mean Rating', 'cost'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in matches:
        df_new = df_new.append(pd.DataFrame(temp[['cuisines','Mean Rating', 'cost']][temp.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['cuisines','Mean Rating', 'cost'], keep=False)
    df_new = df_new.sort_values(by='Mean Rating', ascending=False).head(10)
    
    print('TOP 10 RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % name))
    
    return df_new

SyntaxError: unmatched ')' (Temp/ipykernel_12012/3281343544.py, line 19)

In [248]:
new_df.sample(5)

,name,online_order,book_table,rate,location,cuisines,cost,reviews_list,city,Mean Rating
1151,Karama Restaurant,Yes,Yes,4.4,Frazer Town,"Arabian, North Indian, Beverages",750.0,one of the best restaurant in banagalore for a...,Frazer Town,3.868182
2810,HSR Agarval Bhavan,Yes,No,3.2,HSR,"South Indian, Street Food, Fast Food, Desserts...",400.0,heard about this place by the app magicpin and...,Koramangala 6th Block,3.200000
8941,K27 - The Pub,No,No,3.1,Banashankari,"Chinese, North Indian",1000.0,convenient and average prices a neighbourhood...,Jayanagar,3.100000
2020,Moto Store & CafÃÂÃÂÃÂÃÂÃÂÃÂÃÂÃÂ©,Yes,No,4.2,Ulsoor,"Pizza, Italian, Cafe, Burger, Finger Food, Fas...",500.0,a small and neat little cafe opposite ulsoor l...,Indiranagar,4.200000
117,The Terminus - The Oterra,No,Yes,4.0,Electronic City,"Finger Food, Continental",2500.0,such an excellent place bit far from main cit...,Electronic City,4.000000


In [249]:
recommend_rest(indices,"Ambur Hot Dum Biriyani",cosine_similarities)

TOP 10 RESTAURANTS LIKE Ambur Hot Dum Biriyani WITH SIMILAR REVIEWS: 


,cuisines,Mean Rating,cost
Biergarten,"Continental, European, BBQ, Chinese, Asian",4.766667,2400.0
Communiti,"Continental, BBQ, Salad",4.644444,1500.0
Eggzotic,"North Indian, Chinese, Biryani, Fast Food",3.950000,500.0
Vaho Kafe & Pressery,"Cafe, Healthy Food, Asian, Italian, North Indi...",3.900000,500.0
Cinnamon,"North Indian, Chinese, Biryani",3.828571,550.0
B.M.W - Bhookh Mitaane Wala,"North Indian, South Indian, Chinese",3.677778,500.0
Donne Biriyani House,"South Indian, North Indian, Chinese, Biryani",3.635135,400.0
Swad 'E' Punjab,"North Indian, Chinese, Mughlai",3.600000,500.0
Chef in,"Biryani, North Indian, Chinese",3.600000,500.0
The Diner,"North Indian, Chinese, Andhra, Biryani",3.600000,700.0
